<a href="https://colab.research.google.com/github/jhtwiz/AI-1-jhtwiz/blob/main/Chapter2_%EC%8B%AC%ED%99%94%EA%B3%BC%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GPT fine-tuning으로 뉴스 기사 분류 모델 학습하기


In [ ]:
!pip install tqdm boto3 requests regex sentencepiece sacremoses datasets

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.1/139.1 kB 823.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.7 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully

In [ ]:
import torch
from datasets import load_dataset
from torch.utils.data import DataLoader
from transformers import OpenAIGPTTokenizer

tokenizer = OpenAIGPTTokenizer.from_pretrained("openai-gpt")

tokenizer.pad_token = tokenizer.unk_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/816k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/458k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/656 [00:00<?, ?B/s]

ftfy or spacy is not installed using BERT BasicTokenizer instead of SpaCy & ftfy.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
ds = load_dataset("fancyzhx/ag_news")
len_classes = len(ds['train'].features['label'].names)

def collate_fn(batch):
  texts, labels = [], []
  for row in batch:
    labels.append(row['label'])
    texts.append(row['text'])

  tokenizedText = tokenizer(texts, padding='longest', return_tensors="pt")

  texts = tokenizedText['input_ids']
  attention_mask = tokenizedText['attention_mask']
  labels = torch.LongTensor(labels)

  return texts, attention_mask, labels


train_loader = DataLoader(
    ds['train'], batch_size=64, shuffle=True, collate_fn=collate_fn
)
test_loader = DataLoader(
    ds['test'], batch_size=64, shuffle=False, collate_fn=collate_fn
)

README.md:   0%|          | 0.00/8.07k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

In [ ]:
data = iter(train_loader)
texts, _, labels = next(data)
print(texts.shape)
print(texts[0])
print(labels[0])
print(ds['train'].features['label'].names)
print(ds['train'].features['label'].names[labels[0]])

torch.Size([64, 189])
tensor([15697, 13263, 11427,  9982,   240, 32155,  5223,  1950,     7,  7823,
         2788,   240, 15697,   276, 10689,  1694,   275,   260,  2074, 32155,
         1407,  8876, 15697,   504,  5498,   861,   481,  3138,   256,   252,
         9563, 15503, 16269,   500,  3479,  1218,  2112,   491,  1423,  9982,
          989,   488,  6291,   803,   277,   240, 16708,   557,  4644,  5682,
          240,  7210, 15737,   488,  8466,  5752, 10522,   239,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,

In [ ]:
from transformers import OpenAIGPTModel
model = OpenAIGPTModel.from_pretrained("openai-gpt")
model

model.safetensors:   0%|          | 0.00/479M [00:00<?, ?B/s]

OpenAIGPTModel(
  (tokens_embed): Embedding(40478, 768)
  (positions_embed): Embedding(512, 768)
  (drop): Dropout(p=0.1, inplace=False)
  (h): ModuleList(
    (0-11): 12 x Block(
      (attn): Attention(
        (c_attn): Conv1D()
        (c_proj): Conv1D()
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (mlp): MLP(
        (c_fc): Conv1D()
        (c_proj): Conv1D()
        (act): NewGELUActivation()
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    )
  )
)

DistilBERT 와는 완전히 다른 구조를 갖고 있다. Layer 수는 2배 많고 Attention에서 Linear가 아닌 Conv1D인 점, activation 함수 등 너무나 많은 차이점을 보인다.

In [ ]:
from torch import nn
from transformers import OpenAIGPTModel

class NewsClassifier(nn.Module):
  def __init__(self, n_class):
    super().__init__()

    self.encoder = OpenAIGPTModel.from_pretrained("openai-gpt") # openai-gpt Encoder 사용
    # Encoder Freeze
    for param in self.encoder.parameters():
      param.requires_grad = False
    self.classifier = nn.Linear(768, n_class)

  def forward(self, x, attention_mask=None):
    x = self.encoder(x, attention_mask)['last_hidden_state']
    batch_size = x.shape[0]

    # GPT는 순차적으로 학습하기 때문에 마지막 토큰에 이전 토큰의 데이터들이 담겨 있다고 이해함. 그렇기 때문에 마지막 토큰을 layer에 넣어야 할 것으로 보임.
    # 하지만 [:, -1]은 sequnce에 padding으로 채운 unk_token을 잡을 수 있기 때문에 -1이 아닌 실제 토큰 중 마지막 토큰을 넣는게 좋지 않을까?
    if attention_mask is not None:
      last_valid_token_idx = attention_mask.sum(dim=1) - 1 # 실제 토큰의 길이(= 마지막 토큰 위치)

      # 각 배치의 실제 마지막 토큰
      x = x[torch.arange(batch_size), last_valid_token_idx]
    else:
      x = x[:, -1]  # attention_mask가 없으면 마지막 토큰을 사용

    x = self.classifier(x)

    return x


model = NewsClassifier(len_classes)

In [ ]:
# 이부분은 NewsClassifier init으로 이동
# for param in model.encoder.parameters():
#   param.requires_grad = False

In [ ]:
from torch.optim import Adam
import numpy as np
import matplotlib.pyplot as plt


lr = 0.001
model = model.to('cuda')
loss_fn = nn.CrossEntropyLoss()

optimizer = Adam(model.parameters(), lr=lr)
n_epochs = 10

for epoch in range(n_epochs):
  total_loss = 0.
  model.train()
  for data in train_loader:
    optimizer.zero_grad()
    inputs, attention_mask, labels = data
    inputs, attention_mask, labels = inputs.to('cuda'), attention_mask.to('cuda'), labels.to('cuda').long()

    preds = model(inputs, attention_mask)
    loss = loss_fn(preds, labels)
    loss.backward()
    optimizer.step()

    total_loss += loss.item()

  print(f"Epoch {epoch:3d} | Train Loss: {total_loss}")

Epoch   0 | Train Loss: 821.1520210504532
Epoch   1 | Train Loss: 748.522578984499
Epoch   2 | Train Loss: 739.3776644915342
Epoch   3 | Train Loss: 734.1761130541563
Epoch   4 | Train Loss: 735.1792914271355
Epoch   5 | Train Loss: 735.2640056461096
Epoch   6 | Train Loss: 732.9132820367813
Epoch   7 | Train Loss: 732.7610174268484
Epoch   8 | Train Loss: 733.7136073857546
Epoch   9 | Train Loss: 729.5759753584862


In [ ]:
def accuracy(model, dataloader):
  cnt = 0
  acc = 0

  for data in dataloader:
    inputs, attention_mask, labels = data
    inputs, attention_mask, labels = inputs.to('cuda'), attention_mask.to('cuda'), labels.to('cuda').long()

    preds = model(inputs, attention_mask)
    preds = torch.argmax(preds, dim=-1)

    cnt += labels.shape[0]
    acc += (labels == preds).sum().item()

  return acc / cnt


with torch.no_grad():
  model.eval()
  train_acc = accuracy(model, train_loader)
  test_acc = accuracy(model, test_loader)
  print(f"=========> Train acc: {train_acc:.3f} | Test acc: {test_acc:.3f}")

=========> Train acc: 0.870 | Test acc: 0.865


**openai-gpt(GPT-1)의 layer수가 많아서 그런건지 Linear와 Conv1D의 차이 때문인지 동일 Embedding차원, 동일 데이터인데 학습하는 시간이 조금 더 오래 걸렸던 것 같은 느낌**

**성능의 차이는 GPT-1이 텍스트 생성에 주 목적을 가진 모델이라 그것을 fine-tuning해서 사용한다고 해도 전체 문맥을 학습하는 BERT류에 못 미치는게 아닐까? 라는 생각**

**GPT-2는 어떨까?**

In [ ]:
import torch
from datasets import load_dataset
from torch.utils.data import DataLoader
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

tokenizer.pad_token = tokenizer.unk_token

ds = load_dataset("fancyzhx/ag_news")
len_classes = len(ds['train'].features['label'].names)

def collate_fn(batch):
    texts, labels = [], []
    for row in batch:
        labels.append(row['label'])
        texts.append(row['text'])

    tokenizedText = tokenizer(texts, padding='longest', return_tensors="pt")

    texts = tokenizedText['input_ids']
    attention_mask = tokenizedText['attention_mask']
    labels = torch.LongTensor(labels)

    return texts, attention_mask, labels


train_loader = DataLoader(
    ds['train'], batch_size=64, shuffle=True, collate_fn=collate_fn
)
test_loader = DataLoader(
    ds['test'], batch_size=64, shuffle=False, collate_fn=collate_fn
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

README.md:   0%|          | 0.00/8.07k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

In [ ]:
from transformers import GPT2Model

gpt2 = GPT2Model.from_pretrained('gpt2')
gpt2

GPT2Model(
  (wte): Embedding(50257, 768)
  (wpe): Embedding(1024, 768)
  (drop): Dropout(p=0.1, inplace=False)
  (h): ModuleList(
    (0-11): 12 x GPT2Block(
      (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): GPT2SdpaAttention(
        (c_attn): Conv1D()
        (c_proj): Conv1D()
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (mlp): GPT2MLP(
        (c_fc): Conv1D()
        (c_proj): Conv1D()
        (act): NewGELUActivation()
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
)

구조는 GPT-1과 유사한 듯한 모습. 무슨 차이가 있는지는 추후에 더 확인해 볼 예정

In [ ]:
from torch import nn
from transformers import GPT2Model

class GPT2NewsClassifier(nn.Module):
  def __init__(self, n_class):
    super().__init__()

    self.encoder = GPT2Model.from_pretrained('gpt2')
    # Encoder Freeze
    for param in self.encoder.parameters():
      param.requires_grad = False
    self.classifier = nn.Linear(768, n_class)

  def forward(self, x, attention_mask=None):
    #gpt2는 두번째 파라미터가 attention_mask가 아니라 parameter key값 추가
    x = self.encoder(x, attention_mask=attention_mask)['last_hidden_state']
    batch_size = x.shape[0]

    if attention_mask is not None:
      last_valid_token_idx = attention_mask.sum(dim=1) - 1
      x = x[torch.arange(batch_size), last_valid_token_idx]
    else:
      x = x[:, -1]

    x = self.classifier(x)

    return x


gpt2_model = GPT2NewsClassifier(len_classes)

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

In [ ]:
from torch.optim import Adam
import numpy as np
import matplotlib.pyplot as plt

#리소스가 없어서 3 epoch만....
lr = 0.001
gpt2_model = gpt2_model.to('cuda')
loss_fn = nn.CrossEntropyLoss()

gpt2_optimizer = Adam(gpt2_model.parameters(), lr=lr)
n_epochs = 3

for epoch in range(n_epochs):
  total_loss = 0.
  gpt2_model.train()
  for data in train_loader:
    gpt2_optimizer.zero_grad()
    inputs, attention_mask, labels = data
    inputs, attention_mask, labels = inputs.to('cuda'), attention_mask.to('cuda'), labels.to('cuda').long()

    preds = gpt2_model(inputs, attention_mask)
    loss = loss_fn(preds, labels)
    loss.backward()
    gpt2_optimizer.step()

    total_loss += loss.item()

  print(f"Epoch {epoch:3d} | Train Loss: {total_loss}")

Epoch   0 | Train Loss: 683.0118822157383
Epoch   1 | Train Loss: 546.2753101959825
Epoch   2 | Train Loss: 534.5920732095838


In [ ]:
def accuracy(model, dataloader):
  cnt = 0
  acc = 0

  for data in dataloader:
    inputs, attention_mask, labels = data
    inputs, attention_mask, labels = inputs.to('cuda'), attention_mask.to('cuda'), labels.to('cuda').long()

    preds = model(inputs, attention_mask)
    preds = torch.argmax(preds, dim=-1)

    cnt += labels.shape[0]
    acc += (labels == preds).sum().item()

  return acc / cnt


with torch.no_grad():
  gpt2_model.eval()
  train_acc = accuracy(gpt2_model, train_loader)
  test_acc = accuracy(gpt2_model, test_loader)
  print(f"=========> Train acc: {train_acc:.3f} | Test acc: {test_acc:.3f}")

=========> Train acc: 0.904 | Test acc: 0.903


3Epoch에 준수한 정확도를 보임. 이전에 진행했던 과제에서 빠른 overfitting을 확인했기에 조금 더 학습해서 다시 확인

In [ ]:
from torch.optim import Adam
import numpy as np
import matplotlib.pyplot as plt

#4epoch 더
lr = 0.001
gpt2_model = gpt2_model.to('cuda')
loss_fn = nn.CrossEntropyLoss()

gpt2_optimizer = Adam(gpt2_model.parameters(), lr=lr)
n_epochs = 4

for epoch in range(n_epochs):
  total_loss = 0.
  gpt2_model.train()
  for data in train_loader:
    gpt2_optimizer.zero_grad()
    inputs, attention_mask, labels = data
    inputs, attention_mask, labels = inputs.to('cuda'), attention_mask.to('cuda'), labels.to('cuda').long()

    preds = gpt2_model(inputs, attention_mask)
    loss = loss_fn(preds, labels)
    loss.backward()
    gpt2_optimizer.step()

    total_loss += loss.item()

  print(f"Epoch {epoch:3d} | Train Loss: {total_loss}")

Epoch   0 | Train Loss: 531.2206222042441
Epoch   1 | Train Loss: 531.2194737121463
Epoch   2 | Train Loss: 527.7787817791104
Epoch   3 | Train Loss: 527.9816609509289


In [ ]:
with torch.no_grad():
  gpt2_model.eval()
  train_acc = accuracy(gpt2_model, train_loader)
  test_acc = accuracy(gpt2_model, test_loader)
  print(f"=========> Train acc: {train_acc:.3f} | Test acc: {test_acc:.3f}")

=========> Train acc: 0.902 | Test acc: 0.895


3Epoch 때와 유사한 결과를 보인다. GPT-2랑 DistilBERT가 GPT-1 보다 뉴스 분류에는 더 적합한 것이라 판단.